![logog](https://raw.githubusercontent.com/Pacific-AI-Corp/langtest/main/docs/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Pacific-AI-Corp/langtest/blob/main/demo/tutorials/misc/HF_Callback_NER.ipynb)

**LangTest** is an open-source python library designed to help developers deliver safe and effective Natural Language Processing (NLP) models. Whether you are using **John Snow Labs, Hugging Face, Spacy** models or **OpenAI, Cohere, AI21, Hugging Face Inference API and Azure-OpenAI** based LLMs, it has got you covered. You can test any Named Entity Recognition (NER), Text Classification, fill-mask, Translation model using the library. We also support testing LLMS for Question-Answering, Summarization and text-generation tasks on benchmark datasets. The library supports 60+ out of the box tests. For a complete list of supported test categories, please refer to the [documentation](http://langtest.org/docs/pages/docs/test_categories).

Metrics are calculated by comparing the model's extractions in the original list of sentences against the extractions carried out in the noisy list of sentences. The original annotated labels are not used at any point, we are simply comparing the model against itself in a 2 settings.

# Getting started with LangTest

In [ ]:
!pip install "langtest[transformers, evaluate]"

# LangTestCallback and Its Parameters

The LangTestCallback class is a testing class for Natural Language Processing (NLP) models. It evaluates the performance of a NLP model on a given task using test data and generates a report with test results. It can be imported from the LangTest library in the following way.

In [3]:
#Import Harness from the LangTest library
from langtest.callback import LangTestCallback

It imports the callback class from within the module, that is designed to provide a blueprint or framework for conducting NLP testing, and instances of the callback class can be customized or configured for different testing scenarios or environments then provided to the trainer.

Here is a list of the different parameters that can be passed to the LangTestCallback function:

<br/>

| Parameter             | Description |
| --------------------- | ----------- |
| **task**              | Task for which the model is to be evaluated (text-classification or ner) |
| **data**              | The data to be used for evaluation. A dictionary providing flexibility and options for data sources. It should include the following keys: <ul><li>data_source (mandatory): The source of the data.</li><li>subset (optional): The subset of the data.</li><li>feature_column (optional): The column containing the features.</li><li>target_column (optional): The column containing the target labels.</li><li>split (optional): The data split to be used.</li><li>source (optional): Set to 'huggingface' when loading Hugging Face dataset.</li></ul> |
| **config**            | Configuration for the tests to be performed, specified in the form of a YAML file. |
| **print_reports**     | A bool value that specifies if the reports should be printed. |
| **save_reports**      | A bool value that specifies if the reports should be saved. |
| **run_each_epoch**    | A bool value that specifies if the tests should be run after each epoch or the at the end of training |

<br/>
<br/>

## Preparing for training

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install tensorflow -U

In [ ]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/langtest/main/demo/data/conll03.conll
!wget https://raw.githubusercontent.com/JohnSnowLabs/langtest/main/langtest/data/conll/sample.conll

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertForTokenClassification, BertTokenizerFast, Trainer, TrainingArguments
from seqeval.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
file_path = "conll03.conll"

def read_conll_file(file_path):
    with open(file_path, "r") as file:
        lines = file.readlines()
    return lines[::2]

lines = read_conll_file(file_path)

# Preprocess dataset
def preprocess_conll(lines):
    tokens = []
    labels = []
    token_list = []
    label_list = []

    for line in lines:
        if line.startswith("-DOCSTART-") or line == "\n":
            if token_list:
                tokens.append(token_list)
                labels.append(label_list)
            token_list = []
            label_list = []
        else:
            token, _, _, label = line.strip().split()
            token_list.append(token)
            label_list.append(label)

    return tokens, labels

tokens, labels = preprocess_conll(lines)

class NERDataset(Dataset):
    def __init__(self, tokens, labels, tokenizer, max_length=128):
        self.tokens = tokens
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

        self.label_map = {label: i for i, label in enumerate(sorted(set([lbl for doc_labels in labels for lbl in doc_labels])))}
        self.id2label = {v: k for k, v in self.label_map.items()}

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        token_list = self.tokens[idx]
        label_list = self.labels[idx]

        encoded = self.tokenizer(token_list, is_split_into_words=True, padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt")
        token_ids = encoded.input_ids.squeeze(0)
        attention_mask = encoded.attention_mask.squeeze(0)

        label_ids = [self.label_map[label] for label in label_list]
        label_ids = [-100] + label_ids + [-100]  # Account for [CLS] and [SEP] tokens
        label_ids += [-100] * (self.max_length - len(label_ids))  # Pad labels

        return {
            "input_ids": token_ids,
            "attention_mask": attention_mask,
            "labels": torch.tensor(label_ids, dtype=torch.long),
        }

# Initialize tokenizer and dataset
tokenizer = BertTokenizerFast.from_pretrained("dslim/bert-base-NER")
train_dataset = NERDataset(tokens, labels, tokenizer)

# Initialize model
model = BertForTokenClassification.from_pretrained(
    "dslim/bert-base-NER",
    num_labels=len(train_dataset.label_map),
    id2label=train_dataset.id2label,
    label2id=train_dataset.label_map,
    ignore_mismatched_sizes=True,
)

# Initialize the classifier layer with the correct number of labels
model.classifier = torch.nn.Linear(model.config.hidden_size, len(train_dataset.label_map))

# Move the model to the appropriate device
model.to(device)

## Creating a LangTestCallback instance

After loading the model and tokenizer from huggingface, we can get to the training part of our process. We will utilize `transformers.Trainer` for easily integrating our callback into the training process. We will also use `transformers.TrainingArguments` to specify the training arguments.

We can store the config in a dictionary and pass it to the LangTestCallback function for easier use and visual appeal. The config will be used in this notebook is below:

In [8]:
config = {
 "tests": {
  "defaults": {
   "min_pass_rate": 1.0
  },
  "robustness": {
   "add_typo": {"min_pass_rate": 0.7},
   "uppercase": {"min_pass_rate": 0.7},
   "american_to_british": {"min_pass_rate": 0.7},
  },
  "accuracy": {
   "min_micro_f1_score": {
    "min_score": 0.7
   }
  },
  "bias": {
   "replace_to_female_pronouns": {
    "min_pass_rate": 0.7
   },
   "replace_to_low_income_country": {
    "min_pass_rate": 0.7
   }
  }
 }
}
my_callback = LangTestCallback(task="ner", data={"data_source":"sample.conll"}, config=config, save_reports=True, run_each_epoch=True)

## Creating the Trainer

As mentioned earlier, we create a TrainingArguments object to specify the training arguments. We will also create a Trainer object to train our model. Then we can pass the LangTestCallback object to the Trainer object as a callback. LangTestCallback initilizes the harness object and generates the testcases using .generate() after the trainer is initialized.

In [10]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=64,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=1000,
    learning_rate=3e-5,
    weight_decay=0.01,
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    callbacks=[my_callback],
)

Test Configuration : 
 {
 "tests": {
  "defaults": {
   "min_pass_rate": 1.0
  },
  "robustness": {
   "add_typo": {
    "min_pass_rate": 0.7
   },
   "uppercase": {
    "min_pass_rate": 0.7
   },
   "american_to_british": {
    "min_pass_rate": 0.7
   }
  },
  "accuracy": {
   "min_micro_f1_score": {
    "min_score": 0.7
   }
  },
  "bias": {
   "replace_to_female_pronouns": {
    "min_pass_rate": 0.7
   },
   "replace_to_low_income_country": {
    "min_pass_rate": 0.7
   }
  }
 }
}


## Training

The actual training step is very simple. We just need to call the train() method of the Trainer object. We can also pass the training arguments to the train() method but its default values are OK in this case.

We have the reports printed and also saved under the reports folder. The reports are saved in the form of a MD file. The reports folder is created in the same directory as the notebook.We have the reports printed and also saved under the reports folder. The reports are saved in the form of a MD file. The reports folder is created in the same directory as the notebook.

In [11]:
# Training the model
trainer.train()

Generating testcases...: 100%|██████████| 3/3 [00:00<?, ?it/s]
[W010] - Test 'add_typo': 14 samples removed out of 226
[W010] - Test 'uppercase': 28 samples removed out of 226
[W010] - Test 'american_to_british': 226 samples removed out of 226

[W010] - Test 'replace_to_female_pronouns': 198 samples removed out of 226
[W010] - Test 'replace_to_low_income_country': 138 samples removed out of 226

Running testcases... : 100%|██████████| 527/527 [01:33<00:00,  5.62it/s]


,category,test_type,fail_count,pass_count,pass_rate,minimum_pass_rate,pass
0,robustness,add_typo,65,147,69%,70%,False
1,robustness,uppercase,129,69,35%,70%,False
2,accuracy,min_micro_f1_score,0,1,100%,100%,True
3,bias,replace_to_female_pronouns,11,17,61%,70%,False
4,bias,replace_to_low_income_country,44,44,50%,70%,False


Running testcases... : 100%|██████████| 527/527 [00:13<00:00, 39.39it/s]


,category,test_type,fail_count,pass_count,pass_rate,minimum_pass_rate,pass
0,robustness,add_typo,65,147,69%,70%,False
1,robustness,uppercase,129,69,35%,70%,False
2,accuracy,min_micro_f1_score,0,1,100%,100%,True
3,bias,replace_to_female_pronouns,11,17,61%,70%,False
4,bias,replace_to_low_income_country,44,44,50%,70%,False


{'train_runtime': 2029.1679, 'train_samples_per_second': 1.67, 'train_steps_per_second': 0.027, 'train_loss': 0.7498808260317202, 'epoch': 2.0}


TrainOutput(global_step=54, training_loss=0.7498808260317202, metrics={'train_runtime': 2029.1679, 'train_samples_per_second': 1.67, 'train_steps_per_second': 0.027, 'train_loss': 0.7498808260317202, 'epoch': 2.0})